### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=20130101&end_date=20230531&fl=headline%2Cweb_url%2Csnippet%2Csource%2Ckeywords%2Cpub_date%2Cbyline%2Cword_count&fq=section_name%3A%22Movies%22%20AND%20type_of_material%3A%22Review%22%20AND%20headline%3A%22love%22&sort=newest&api-key={nyt_api_key}"

In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):

    # create query with a page number
    # API results show 10 articles at a time
    query_url_page = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url_page).json()

    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)

    
    # Try and save the reviews to the reviews_list
    try:

        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Checked page {page}")

        # Print the page number that had no results then break from the loop
    except:
        print(f"No results for page {page}")
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)

reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody L

In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].to_list()
print(titles)


['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story'

### Access The Movie Database API

In [11]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = str("&api_key=" + tmdb_api_key)

In [14]:
print(titles)
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for i in range(len(titles)):
    title = str(titles[i])
    print(f"Checking for movie: {title}")
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(1)
        print("Sleeping for 1 second")

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response = requests.get(url + title + tmdb_key_string).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = response["results"][0]["id"]

        # Make a request for a the full movie details
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        movie = requests.get(movie_url).json()
        
        # Extract the genre names into a list
        genres = movie["genres"]

        # Extract the spoken_languages' English name into a list
        spoken_languages = movie["spoken_languages"]

        # Extract the production_countries' name into a list
        production_countries = movie["production_countries"]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie["title"],
            "original_title": movie["original_title"],
            "budget": movie["budget"],
            "original_language": movie["original_language"],
            "homepage": movie["homepage"],
            "overview": movie["overview"],
            "popularity": movie["popularity"],
            "runtime": movie["runtime"],
            "revenue": movie["revenue"],
            "release_date": movie["release_date"],
            "vote_average": movie["vote_average"],
            "vote_count": movie["vote_count"],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        })
        
        # Print out the title that was found
        print(f"Found movie: {movie['title']}")
    except:
        # Print out the title that was not found
        print(f"Movie not found: {title}")


['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan', 'West Side Story'

In [17]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))


[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 1.518,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genres": [
            {
                "id": 18,
                "name": "Drama"
            },
            {
                "id": 9648,
                "name": "Mystery"
            },
            {
                "id": 53,
              

In [18]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df.head()


,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",1.518,102,0,2021-10-07,3.000,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",35.402,96,15055,2023-03-24,6.700,33,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",21.446,96,0,2023-04-21,6.300,158,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",11.067,104,84178,2022-09-21,6.842,184,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'FR', 'name': 'France'}]"
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,16.100,100,37820,2023-04-07,6.412,68,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."


### Merge and Clean the Data for Export

In [29]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_movies_df, on="title")
merged_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.518,102,0,2021-10-07,3.000,4,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...","[{'english_name': 'Spanish', 'iso_639_1': 'es'...","[{'iso_3166_1': 'AR', 'name': 'Argentina'}]"
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",35.402,96,15055,2023-03-24,6.700,33,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso..."
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,16.100,100,37820,2023-04-07,6.412,68,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'..."
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,3.909,95,77145,2023-04-13,6.000,2,"[{'id': 99, 'name': 'Documentary'}]","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,12.500,116,3395595,2023-03-31,6.867,83,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...","[{'english_name': 'English', 'iso_639_1': 'en'...","[{'iso_3166_1': 'US', 'name': 'United States o..."


In [20]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
fix_list = ["genres", "spoken_languages", "production_countries"]

# Create a list of characters to remove
chars_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in fix_list:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
        # Remove the character from the column
        merged_df[column] = merged_df[column].str.replace(char, "")

# Display the fixed DataFrame
merged_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",1.518,102,0,2021-10-07,3.000,4,"{id: 18, name: Drama}, {id: 9648, name: Myster...","{english_name: Spanish, iso_639_1: es, name: E...","{iso_3166_1: AR, name: Argentina}"
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",35.402,96,15055,2023-03-24,6.700,33,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: CA, name: Canada}, {iso_3166_1: U..."
2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,16.100,100,37820,2023-04-07,6.412,68,"{id: 10749, name: Romance}, {id: 18, name: Dra...","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: CZ, name: Czech Republic}, {iso_3..."
3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,3.909,95,77145,2023-04-13,6.000,2,"{id: 99, name: Documentary}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: US, name: United States of America}"
4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,...,Struggling but unapologetically living on her ...,12.500,116,3395595,2023-03-31,6.867,83,"{id: 18, name: Drama}, {id: 80, name: Crime}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: US, name: United States of America}"


In [21]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])


In [22]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index()
merged_df.head()

,index,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,...,"Argentina, 1970s. A desperate young woman goes...",1.518,102,0,2021-10-07,3.000,4,"{id: 18, name: Drama}, {id: 9648, name: Myster...","{english_name: Spanish, iso_639_1: es, name: E...","{iso_3166_1: AR, name: Argentina}"
1,1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,...,"When Jaime, a gay teenager, is sent to live in...",35.402,96,15055,2023-03-24,6.700,33,"{id: 18, name: Drama}, {id: 10749, name: Romance}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: CA, name: Canada}, {iso_3166_1: U..."
2,2,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,...,Emma and Jesse are living the perfect life tog...,16.100,100,37820,2023-04-07,6.412,68,"{id: 10749, name: Romance}, {id: 18, name: Dra...","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: CZ, name: Czech Republic}, {iso_3..."
3,3,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,...,May Pang lovingly recounts her life in rock & ...,3.909,95,77145,2023-04-13,6.000,2,"{id: 99, name: Documentary}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: US, name: United States of America}"
4,4,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,subject: Movies;creative_works: A Thousand and...,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,...,Struggling but unapologetically living on her ...,12.500,116,3395595,2023-03-31,6.867,83,"{id: 18, name: Drama}, {id: 80, name: Crime}","{english_name: English, iso_639_1: en, name: E...","{iso_3166_1: US, name: United States of America}"


In [31]:
# Export data to CSV without the index
merged_df.to_csv("./Finished_data.csv", index=False)
